In [3]:
import random
import pandas as pd
import numpy as np
import os
import glob
import tensorflow as tf
from tensorflow.data import Dataset
import tensorflow.keras as keras
from tqdm.auto import tqdm
datapath='./drive/MyDrive/project'

In [4]:
all_input_list = sorted(glob.glob(datapath+'/train_input/*.csv'))
all_target_list = sorted(glob.glob(datapath+'/train_target/*.csv'))
train_input_list = all_input_list[:25]
train_target_list = all_target_list[:25]
val_input_list = all_input_list[25:]
val_target_list = all_target_list[25:]

<h2>데이터 fetch, Min-Max scaling, concat </h2>

In [28]:

def CustomDataset(input_paths, target_paths,tradition=False):
  data_list = []
  label_list = []
  input_data=pd.DataFrame()
  target_data=pd.DataFrame()
  print('Data Pre-processing..')
  for input_path, target_path in tqdm(zip(input_paths, target_paths)):
    input_df = pd.read_csv(input_path)
    target_df = pd.read_csv(target_path)
    input_df = input_df.drop(columns=['obs_time'])          
    input_length = int(len(input_df)/24)
    target_length = int(len(target_df))
    for idx in range(target_length):
      temp = input_df[24*idx:24*(idx+1)]
      for i in [6,8,10,12,14]:
        if temp.iloc[23][i]==0.0:
          continue
        else:
          temp.iloc[:,i]=temp.iloc[:,i]/temp.iloc[23][i]
      input_df[24*idx:24*(idx+1)]=temp
    input_data=pd.concat([input_data,input_df])
    target_data=pd.concat([target_data,target_df])
  min=np.array(input_data.min()).reshape(1,-1)
  max=np.array(input_data.max()).reshape(1,-1)
  for input_path, target_path in tqdm(zip(input_paths, target_paths)):
    input_df = pd.read_csv(input_path)
    target_df = pd.read_csv(target_path)
    input_df = input_df.drop(columns=['obs_time'])          
    input_length = int(len(input_df)/24)
    target_length = int(len(target_df))
    for idx in range(target_length):
      time_series = input_df[24*idx:24*(idx+1)].values
      for i in [6,8,10,12,14]:
        if time_series[-1][i]==0.0:
          continue
        else:
          time_series[:,i]=time_series[:,i]/time_series[-1][i]
      data_list.append(tf.convert_to_tensor((time_series-min)/(max-min)))
    for label in target_df["predicted_weight_g"]:
      label_list.append(label)
  if tradition:
    return input_data,target_data
  else:
    return data_list,label_list

  

In [29]:
#input으로 target의 train의 path, target의 path를 문자열 리스트 형태로 넣습니다.
data,target=CustomDataset(train_input_list,train_target_list)

Data Pre-processing..


0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [34]:
tf.convert_to_tensor(data).shape #하루치 데이터를 2-D 형태로 넣고, 이이 2D 텐서가 700개 있습니다. 

TensorShape([700, 24, 15])